In [1]:
import itertools
import networkx as nx
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesCard')
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

In [2]:
import numpy as np
import pandas as pd
import time
import sys
import copy
from Models.pgmpy_BN import Pgmpy_BN
from Testing.toy_dataset import *

In [3]:
class VariableElimination(object):
    def __init__(self, model):
        self.model = model
        model.check_model()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
        self.root = self.get_root()

        
    def get_root(self):
        """Returns the network's root node."""
        def find_root(graph, node):
            predecessor = next(self.model.predecessors(node), None)
            if predecessor:
                root = find_root(graph, predecessor)
            else:
                root = node
            return root

        return find_root(self, list(self.model.nodes)[0])
    
    
    def steiner_tree(self, nodes):
        """Returns the minimal part of the tree that contains a set of nodes."""
        sub_nodes = set()

        def walk(node, path):

            if len(nodes) == 0:
                return

            if node in nodes:
                sub_nodes.update(path + [node])
                nodes.remove(node)

            for child in self.model.successors(node):
                walk(child, path + [node])
        walk(self.root, [])
        sub_graph = self.model.subgraph(sub_nodes)
        sub_graph.cardinalities = defaultdict(int)
        for node in sub_graph.nodes:
            sub_graph.cardinalities[node] = self.model.cardinalities[node]
        return sub_graph
    
    
    def _get_working_factors(self, variables, evidence):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """
        useful_var = variables + list(evidence.keys())
        sub_graph_model = self.steiner_tree(useful_var)
        variables_sub_graph = set(sub_graph_model.nodes)
        
        working_factors = dict()
        for node in sub_graph_model.nodes:
            working_factors[node] = set()
            for factor in self.factors[node]:
                if set(factor.variables).issubset(variables_sub_graph):
                    working_factors[node].add((factor, None))

        # Dealing with evidence. Reducing factors over it before VE is run.
        if evidence:
            for evidence_var in evidence:
                for factor, origin in working_factors[evidence_var]:
                    factor_reduced = factor.reduce(
                        [(evidence_var, evidence[evidence_var])], inplace=False
                    )
                    for var in factor_reduced.scope():
                        if var in working_factors:
                            working_factors[var].remove((factor, origin))
                            working_factors[var].add((factor_reduced, evidence_var))
                if type(evidence[evidence_var]) != list:
                    del working_factors[evidence_var]
        return working_factors, sub_graph_model
    
    def _get_elimination_order(
        self, variables, evidence, model=None, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        if model is None:
            model = self.model
        if isinstance(model, JunctionTree):
            all_variables = set(chain(*model.nodes()))
        else:
            all_variables = model.nodes()
        
        not_evidence_eliminate = []
        if evidence is not None:
            for key in evidence:
                if type(evidence[key]) != list:
                    not_evidence_eliminate.append(key)
        to_eliminate = (
            set(all_variables)
            - set(variables)
            - set(not_evidence_eliminate)
        )

        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
            not isinstance(elimination_order, str)
        ):
            if any(
                var in elimination_order
                for var in set(variables).union(
                    set(evidence.keys() if evidence else [])
                )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
            model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order
    
    def _variable_elimination(
        self,
        variables,
        operation,
        evidence=None,
        elimination_order="minfill",
        joint=True,
        show_progress=False,
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        tic = time.time()
        working_factors, sub_graph_model = self._get_working_factors(variables, evidence)
        toc = time.time()
        print(f"getting working factors takes {toc-tic} secs")
        elimination_order = self._get_elimination_order(
            variables, evidence, sub_graph_model, elimination_order, show_progress=show_progress
        )
        print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order

        for var in pbar:
            tic = time.time()
            print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            if var == "iYearwrk":
                print(factors)
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                if variable in working_factors:
                    working_factors[variable].add((phi, var))
            eliminated_variables.add(var)
            print(f"eliminating {var} takes {time.time()-tic} secs")
            
        # Step 4: Prepare variables to be returned.
        tic = time.time()
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]
        print(final_distribution)
        print(f"the rest takes {time.time()-tic} secs")
        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            print(f"the rest takes {time.time()-tic} secs")
            return query_var_factor

    def query(
        self,
        variables,
        evidence=None,
        elimination_order="MinFill",
        joint=True,
        show_progress=False,
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
        )

In [4]:
import pickle
with open('check_points/Census_chow-liu.pkl', 'rb') as f:
    BN = pickle.load(f)
with open('check_points/Census_junction.pkl', 'rb') as f:
    BN_J = pickle.load(f)

In [5]:
BN.model.cardinalities = BN.model.get_cardinality()
ve = VariableElimination(BN.model)
factors = ve.factors
cpds = BN.model.cpds

In [6]:
f, s =ve._get_working_factors(["iAvail"], {"iClass": 0, "iKorean": [0,1], "iMay75880": 0, "iRagechld": [0, 4], "iRrelchld": 0})


/Users/ziniuwu/Desktop/research/BayesCard/Pgmpy/factors/discrete/DiscreteFactor.py:281: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Internal error in pre-inference rewriting pass encountered during compilation of function "reduce" due to: Constant inference not possible for: call $42load_global.2($48binary_subscr.5, func=$42load_global.2, args=[Var($48binary_subscr.5, DiscreteFactor.py:306)], kws=(), vararg=None)

File "../Pgmpy/factors/discrete/DiscreteFactor.py", line 306:
    def reduce(self, values, inplace=True):
        <source elided>
            raise TypeError(
                "values: Expected type list of tuples, get type {type}", type(values[0])
                ^

  @jit
/Users/ziniuwu/Desktop/research/BayesCard/Pgmpy/factors/discrete/DiscreteFactor.py:281: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "reduce" failed type inference due to: Untyped global name 'isinstance

/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/site-packages/numba/core/object_mode_passes.py:178: NumbaWarning: Function "reduce" was compiled in object mode without forceobj=True.

File "../Pgmpy/factors/discrete/DiscreteFactor.py", line 318:
    def reduce(self, values, inplace=True):
        <source elided>
        # state numbers.
        for i, (var, state_name) in enumerate(values):
        ^

  state.func_ir.loc))
/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/site-packages/numba/core/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "../Pgmpy/factors/discrete/DiscreteFactor.py", line 318:
    def reduce(self, values, inplace=True):
        <source elided>
        # state n

In [ ]:
print(f.keys())
f

In [11]:
tic = time.time()
q = ve.query(["iAvail"], {"iClass": 0, "iKorean": [0,1], "iMay75880": 0, "iRagechld": [0, 4], "iRrelchld": 0},
            elimination_order="weightedminfill")
print(time.time()-tic)

getting working factors takes 0.06838679313659668 secs
getting elimination orders takes 0.0014400482177734375 secs
iLooking
eliminating iLooking takes 0.0014781951904296875 secs
iKorean
eliminating iKorean takes 0.0003991127014160156 secs
iRagechld
eliminating iRagechld takes 0.00036787986755371094 secs
iRlabor
eliminating iRlabor takes 0.0066530704498291016 secs
iRspouse
eliminating iRspouse takes 0.002586841583251953 secs
iMilitary
eliminating iMilitary takes 0.012909889221191406 secs
iFertil
eliminating iFertil takes 0.0006821155548095703 secs
dAge
eliminating dAge takes 0.0028450489044189453 secs
iRvetserv
eliminating iRvetserv takes 0.0013859272003173828 secs
dIndustry
eliminating dIndustry takes 0.0004630088806152344 secs
dOccup
eliminating dOccup takes 0.00048422813415527344 secs
iYearwrk
[<DiscreteFactor representing phi(iRelat1:14, iYearwrk:8) at 0x1a327aa990>, <DiscreteFactor representing phi(iYearwrk:8) at 0x1a327aa6d0>, <DiscreteFactor representing phi(iAvail:5, iYearwrk:8)

In [ ]:
q.values

In [ ]:
#ve = VariableElimination(BN.model)
BN.infer_machine = ve

In [ ]:
pickle.dump(BN, open('check_points/Census_chow-liu.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
BN.query({"iAvail": 0, "iClass": 0, "iKorean": [0,1], "iMay75880": 0, "iRagechld": [0, 4], "iRrelchld": 0})

In [ ]:
from Evaluation.cardinality_estimation import parse_query_single_table
from time import perf_counter
def evaluate_cardinality(BN, query_path):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        print(f"Predicting cardinality for query {query_no}: {query_str}")
        
        query = parse_query_single_table(query_str.strip(), BN)
        card_start_t = perf_counter()
        print(query)
        try:
            cardinality_predict = BN.query(query)
        except:
            continue
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [ ]:
for i in [50,90,95,99,100]:
    print(np.percentile(q_errors, i))

In [ ]:
np.mean(latencies)